In [1]:
import pandas as pd
import base64
import gzip
import yaml
import re

# Define the path to your configuration file
config_file_path = r'L:\gogomatlab\MATLAB_Exp2_ReadFiles\MaxWell\0YonData\Config_files\13h04m23s.cfg'

# Read the contents of the configuration file
with open(config_file_path, 'r') as f:
    content = f.read()

# Find the index where the second section starts
separator = 'H4sIAAAAAAAC/'
split_index = content.find(separator)

if split_index == -1:
    raise ValueError("Separator 'H4sIAAAAAAAC/' not found in the file.")

# Split the content into two sections
electrode_section = content[:split_index]
config_section = content[split_index:]

# Process the electrode section
entries = electrode_section.strip(';').split(';')

indexes = []
numbers = []
positions_x = []
positions_y = []

# Regular expression pattern to extract data
pattern = r'(\d+)\((\d+)\)([0-9.]+)/([0-9.]+)'

for entry in entries:
    match = re.match(pattern, entry)
    if match:
        idx = int(match.group(1))
        num = int(match.group(2))
        pos_x = float(match.group(3))
        pos_y = float(match.group(4))
        
        indexes.append(idx)
        numbers.append(num)
        positions_x.append(pos_x)
        positions_y.append(pos_y)
    else:
        print(f"Entry '{entry}' is not in expected format")

# Create a DataFrame from the parsed electrode data
electrode_df = pd.DataFrame({
    'index': indexes,
    'number': numbers,
    'position_x': positions_x,
    'position_y': positions_y
})

# Process the configuration section
# Remove any whitespace and newlines
config_data_base64 = ''.join(config_section.strip().split())


# Decode the Base64 data
config_data_bytes = base64.b64decode(config_data_base64)

# Decompress the Gzip data
config_data_decompressed = gzip.decompress(config_data_bytes)

# Decode bytes to string
config_data_str = config_data_decompressed.decode('utf-8')

# Parse the YAML configuration data
config_data_yaml = yaml.safe_load(config_data_str)

# Now, 'electrode_df' contains the electrode data as a DataFrame
# 'config_data_yaml' contains the decompressed configuration data as a dictionary

# For demonstration purposes, print the outputs
print("Electrode Data:")
print(electrode_df)

print("\nConfiguration Data:")
print(yaml.dump(config_data_yaml, default_flow_style=False))


Entry '
' is not in expected format
Electrode Data:
   index  number  position_x  position_y
0     12   11289      1207.5       892.5
1    848   11509      1207.5       910.0
2    860   11290      1225.0       892.5
3    864   11510      1225.0       910.0

Configuration Data:
cmdFillBL1 000000000011111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110000000000
  cmdFillBL2 000000000010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010100000000000
  cmdSelWL 0 cmdSelWL 1 cmdSelWL 137 cmdSelWL 138 cmdFillBL1 00000000001111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

In [18]:
import pandas as pd
import base64
import gzip
import re

# Define the path to your configuration file
# Replace the path with your actual file path
#config_file_path = r'L:\gogomatlab\MATLAB_Exp2_ReadFiles\MaxWell\0YonData\Config_files\2elec_cor.cfg' # two up left electrodes
config_file_path = r'L:\gogomatlab\MATLAB_Exp2_ReadFiles\MaxWell\0YonData\Config_files\2elec_mid.cfg' # two middle electrodes and 2 up left electrodes


def read_config_file(config_file_path):
    # Read the contents of the configuration file
    with open(config_file_path, 'r') as f:
        content = f.read()

    # Find the index where the second section starts
    separator = 'H4sIAAAAAAAC/'
    split_index = content.find(separator)

    if split_index == -1:
        raise ValueError("Separator 'H4sIAAAAAAAC/' not found in the file.")

    # Split the content into two sections
    electrode_section = content[:split_index]
    config_section = content[split_index:]

    return electrode_section, config_section

# Define the electrode_read function before calling it
def electrode_read(electrode_section):
    # Process the electrode section
    entries = electrode_section.strip(';').split(';')

    indexes = []
    numbers = []
    positions_x = []
    positions_y = []

    # Regular expression pattern to extract data
    pattern = r'(\d+)\((\d+)\)([0-9.]+)/([0-9.]+)'

    for entry in entries:
        match = re.match(pattern, entry.strip())
        if match:
            idx = int(match.group(1))
            num = int(match.group(2))
            pos_x = float(match.group(3))
            pos_y = float(match.group(4))
            
            indexes.append(idx)
            numbers.append(num)
            positions_x.append(pos_x)
            positions_y.append(pos_y)
        else:
            print(f"Entry '{entry}' is not in expected format")

    # Create a DataFrame from the parsed electrode data
    electrode_df = pd.DataFrame({
        'index': indexes,
        'number': numbers,
        'position_x': positions_x,
        'position_y': positions_y
    })
    
    return electrode_df  # Return the DataFrame

def extract_config_commands(config_section):
    # Process the configuration section
    # Remove any whitespace and newlines, but preserve necessary characters
    config_data_base64 = ''.join(config_section.strip().split())

    # Calculate required padding
    padding_needed = len(config_data_base64) % 4
    if padding_needed != 0:
        padding = 4 - padding_needed
        config_data_base64 += '=' * padding

    # Decode the Base64 data
    try:
        config_data_bytes = base64.b64decode(config_data_base64)
    except Exception as e:
        print(f"Error decoding Base64 data: {e}")
        exit(1)

    # Decompress the Gzip data
    try:
        config_data_decompressed = gzip.decompress(config_data_bytes)
    except Exception as e:
        print(f"Error decompressing Gzip data: {e}")
        exit(1)

    # Decode bytes to string
    config_data_str = config_data_decompressed.decode('utf-8')

    # Since the configuration data is not in YAML format, parse it as plain text
    # Split the data into lines
    config_lines = config_data_str.strip().split('\n')
    #print(config_lines)  # Print the lines (you can remove this if not needed)

    # split to line following commas
    #config_split_lines = [line.split(',') for line in config_lines]

    #return config_split_lines
    return config_lines
    # print each item in the list
    # for line in config_split_lines:
    #     print(line)

def active_electrode_configure():
    import re

    # Sample commands (Replace with your actual commands)
    commands = [
        # ... (Your provided commands)
    ]

    # Initialize variables
    active_columns = set()
    selected_rows = set()

    # Process the commands
    for cmd in commands:
        cmd = cmd.strip("[]'")  # Clean up the string
        if cmd.startswith('cmdFillBL1'):
            bl1_binary = cmd.split(' ')[1]
            for i, bit in enumerate(bl1_binary):
                if bit == '1':
                    active_columns.add(i)
        elif cmd.startswith('cmdFillBL2'):
            bl2_binary = cmd.split(' ')[1]
            for i, bit in enumerate(bl2_binary):
                if bit == '1':
                    active_columns.add(i + 110)  # Columns 110 to 219
        elif cmd.startswith('cmdSelWL'):
            row_number = int(cmd.split(' ')[1])
            selected_rows.add(row_number)

    # Generate activated electrodes
    activated_electrodes = []
    for row in selected_rows:
        for column in active_columns:
            activated_electrodes.append((row, column))

    # Now, 'activated_electrodes' contains tuples of (row, column) for activated electrodes
    # For demonstration, print the first 10 activated electrodes
    print("First 10 Activated Electrodes:")
    for electrode in activated_electrodes[:10]:
        print(f"Row: {electrode[0]}, Column: {electrode[1]}")

# 
def configuration_encode(commands):
    
    import re

    # Initialize variables
    active_columns = set()
    selected_rows = set()

    # Process the commands
    for cmd in commands:
        cmd = cmd.strip("[]'")  # Clean up the string
        print(cmd)
        if cmd.startswith('cmdFillBL1'):
            bl1_binary = cmd.split(' ')[1]
            for i, bit in enumerate(bl1_binary):
                if bit == '1':
                    active_columns.add(i)
        elif cmd.startswith('cmdFillBL2'):
            bl2_binary = cmd.split(' ')[1]
            for i, bit in enumerate(bl2_binary):
                if bit == '1':
                    active_columns.add(i + 110)  # Columns 110 to 219
        elif cmd.startswith('cmdSelWL'):
            row_number = int(cmd.split(' ')[1])
            selected_rows.add(row_number)

    # Generate activated electrodes
    activated_electrodes = []
    for row in selected_rows:
        for column in active_columns:
            activated_electrodes.append((row, column))

    # Now, 'activated_electrodes' contains tuples of (row, column) for activated electrodes
    # For demonstration, print the first 10 activated electrodes
    print("First 10 Activated Electrodes:")
    for electrode in activated_electrodes[:10]:
        print(f"Row: {electrode[0]}, Column: {electrode[1]}")

    # return the activated electrodes
    return activated_electrodes



# Read and split the configuration file
electrode_section, config_section = read_config_file(config_file_path)

# Process the electrode section
electrode_df = electrode_read(electrode_section)  # Assign the returned DataFrame

# Extract and process the configuration commands
config_split_lines = extract_config_commands(config_section)

# Process the configuration commands to test right configuration decoding
active_electrodes = configuration_encode(config_split_lines)




# # Initialize a list to hold the parsed commands
# config_commands = []
# # Process each line
# for line in config_lines:
#     line = line.strip()
#     print(line)  # Print each line (you can remove this if not needed)
#     if not line:
#         continue  # Skip empty lines

#     # Extract commands and their values
#     cmd_match = re.match(r'(cmd\w+)\s*(.*)', line)
#     if cmd_match:
#         cmd = cmd_match.group(1)
#         value = cmd_match.group(2)
#         config_commands.append((cmd, value))
#     else:
#         # Handle lines that don't match the expected pattern
#         config_commands.append(('unknown', line))

# Now, 'config_commands' is a list of tuples (command, value)
# For demonstration purposes, print the outputs
print("\nElectrode Data:")
print(electrode_df)

# Uncomment the following lines if you want to print the parsed configuration commands
# print("\nParsed Configuration Commands:")
# for cmd, value in config_commands:
#     print(f"{cmd}: {value}")


Entry '
' is not in expected format
cmdFillBL1 000000000011111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110000000000
cmdFillBL2 000000000010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010100000000000
cmdSelWL 0
cmdSelWL 1
cmdSelWL 137
cmdSelWL 138
cmdFillBL1 000000000011111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111101111111111111111111111101111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110000000000
cmdFillBL2 00000000001111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

In [7]:
import re

base64_pattern  = r'(H4sIAAAAAAAC/+1dy5brNg7c5yvyCQXJ8mObRVbZZZEPSGYx59z5/+0kaVsiXgQo+dlUz8ncZrfatk)'
"""SwCBQLwJ//++vX//748ctv9DPmL/rGX8td/vTn7d6H8t6/8f/4vf/+nx9//PZz8S0V346ncnBenhXx
Z/XPEyVC8Xj/favrO5bP/esq8Mv/uf7rb9jV1xeg29/In8u/4Ncv/84fRHwYUtfPP5xfP7KT20eD/h
ww7xPXuy3e+Oua291AX0+3N7r9H5Z35fdb3KdxPeStLve5PKrlwxt2MjgmoGZNvxWxD2ZOgTE71evZ
1GG5ofj6woT0H6ANKoz3gzYJGD+tmsTyF9D3A2+KbZOwTQi1668XaRMYXRSorHrjxr1VXzGJ4kaw3S
T0Bmes5UYTsO5emwSKt6uaBEmUuD0t2K/ftOoXrCmuh/4w2gQOtgn08WWbQD/3/mUC0/Ltcfm2cBfO
hecwXEo3ovQ1RuZtsN3lavLl8oJY32SPP80R/fqutKsbNnx/J/xr+hYjuAgTUI+BIRZHbuhdHmIn4N
fDtx1Y1gWJ7eAfSH7AzKuCFj9SQQsx/5rd9p3HL3rR2XMoQQHKBqBgAMYW2I4JeBGOIA0D6AQGwGGg
fA5yK7jP1H/KVnDFyE62AoYDQ4gDPD4SY8x8B3eT5SugZkpPNq3ucUC4AzRG/sAH328dD3ccuNnAIb
QBGeK2jmszBMVeujOYHXv/Cr/IDAsE5/qNfUKGA9On4oDPa/JfGSyYFxf0i)"""



# Extract the Base64 data using a regular expression
#base64_pattern = r'(H4sIAAAAAAAC/[A-Za-z0-9+/=]+)'
match = re.search(base64_pattern, content)
if match:
    config_data_base64 = match.group(1)
else:
    raise ValueError("Base64 data not found in the file.")
print(config_data_base64)

ValueError: Base64 data not found in the file.